You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

#### Specifically, they have set the following requirements:

- Columns containing categories with only two factors must be stored as Booleans (bool).
- Columns containing integers only must be stored as 32-bit integers.
- Columns containing floats must be stored as 16-bit floats.
- Columns containing nominal categorical data must be stored as the category data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
- The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [6]:
import pandas as pd
from pandas.api.types import CategoricalDtype

ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs.head()

ds_jobs_transformed = ds_jobs.copy()

In [7]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [8]:
# bool conversion
ds_jobs_transformed['relevant_experience'] = ds_jobs['relevant_experience'] == 'Has relevant experience'
ds_jobs_transformed['job_change'] = ds_jobs['job_change'] == 1.0

# int32 conversion 
ds_jobs_transformed['student_id'] = ds_jobs['student_id'].astype('int32')
ds_jobs_transformed['training_hours'] = ds_jobs['training_hours'].astype('int32')

# float16 conversion
ds_jobs_transformed['city_development_index'] = ds_jobs['city_development_index'].astype('float16')

# define ordered categorical types
edu_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
enrollment_order = ['no_enrollment', 'Part time course', 'Full time course']
comp_size_order = ['<10', '10-49', '50-99', '100-499', '500-999', 
                   '1000-4999', '5000-9999', '10000+']
experience_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                    '10', '11', '12', '13', '14', '15', '16', '17', 
                    '18', '19', '20', '>20']
last_job_order = ['never', '1 year', '2 years', '3 years', '4 years', '>4 years']

# apply ordered categoricals with a loop
ordered_columns = {
    'education_level': CategoricalDtype(categories=edu_order, ordered=True),
    'enrolled_university': CategoricalDtype(categories=enrollment_order, ordered=True),
    'company_size': CategoricalDtype(categories=comp_size_order, ordered=True),
    'experience': CategoricalDtype(categories=experience_order, ordered=True),
    'last_new_job': CategoricalDtype(categories=last_job_order, ordered=True)
}

for col, dtype in ordered_columns.items():
    ds_jobs_transformed[col] = ds_jobs[col].astype(dtype)

# apply nominal categoricals with a loop
nominal_columns = ['city', 'gender', 'major_discipline', 'company_type']

for col in nominal_columns:
    ds_jobs_transformed[col] = ds_jobs[col].astype('category')

# filter the new df to include only 10+ years of experience and company size >= 1000 employees
valid_experience = experience_order[experience_order.index('10'):]
valid_sizes = comp_size_order[comp_size_order.index('1000-4999'):]
ds_jobs_transformed = ds_jobs_transformed[
    ds_jobs_transformed['experience'].isin(valid_experience) &
    ds_jobs_transformed['company_size'].isin(valid_sizes)
]

In [9]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            74 non-null     category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)

In [10]:
orig_mem = ds_jobs.memory_usage().sum()
print(f"The original df has roughtly {orig_mem} bytes of memory.")

The original df has roughtly 2145824 bytes of memory.


In [11]:
trans_mem = ds_jobs_transformed.memory_usage().sum()
print(f"The revised df has roughtly {trans_mem} bytes of memory.")

The revised df has roughtly 71213 bytes of memory.


In [14]:
sweet_sweet_efficiency = round((trans_mem / orig_mem) * 100, 2)
reduction = round((1 - trans_mem / orig_mem) * 100)
print(f"Our data is prepped and we are now only using {sweet_sweet_efficiency}% of the initial dataframe size.")

Our data is prepped and we are now only using 3.32% of the initial dataframe size.
